In [16]:
import numpy as np
import random as rm

# THEMES
Add different themes for the system to use, which will control the ranges and values that the musical and timbral parameters can take  
eg. lush, drone, busy, minimal, etc.  



- minimal- One note sustained  
- drone- long sustained chords, sawtooth wave heavy
- lush- full sounding, minimal effects and minimal chord selection
- busy- lots of chords, lots of effects, varying sustain for each note
- voice/choir synth- few chords, not as long sustained as drone/minimal

## Testing

In [2]:
# The statespace
states = ["Sleep","Icecream","Run"]

# Possible sequences of events
transitionName = [["SS","SR","SI"],["RS","RR","RI"],["IS","IR","II"]]

# Probabilities matrix (transition matrix)
transitionMatrix = [[0.2,0.6,0.2],[0.1,0.6,0.3],[0.2,0.7,0.1]]

In [3]:
if sum(transitionMatrix[0])+sum(transitionMatrix[1])+sum(transitionMatrix[1]) != 3:
    print("Somewhere, something went wrong. Transition matrix, perhaps?")
else: print("All is gonna be okay, you should move on!! ;)")


All is gonna be okay, you should move on!! ;)


In [4]:
# A function that implements the Markov model to forecast the state/mood.
def activity_forecast(days):
    # Choose the starting state
    activityToday = "Sleep"
    print("Start state: " + activityToday)
    # Shall store the sequence of states taken. So, this only has the starting state for now.
    activityList = [activityToday]
    i = 0
    # To calculate the probability of the activityList
    prob = 1
    while i != days:
        if activityToday == "Sleep":
            change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix[0])
            if change == "SS":
                prob = prob * 0.2
                activityList.append("Sleep")
                pass
            elif change == "SR":
                prob = prob * 0.6
                activityToday = "Run"
                activityList.append("Run")
            else:
                prob = prob * 0.2
                activityToday = "Icecream"
                activityList.append("Icecream")
        elif activityToday == "Run":
            change = np.random.choice(transitionName[1],replace=True,p=transitionMatrix[1])
            if change == "RR":
                prob = prob * 0.5
                activityList.append("Run")
                pass
            elif change == "RS":
                prob = prob * 0.2
                activityToday = "Sleep"
                activityList.append("Sleep")
            else:
                prob = prob * 0.3
                activityToday = "Icecream"
                activityList.append("Icecream")
        elif activityToday == "Icecream":
            change = np.random.choice(transitionName[2],replace=True,p=transitionMatrix[2])
            if change == "II":
                prob = prob * 0.1
                activityList.append("Icecream")
                pass
            elif change == "IS":
                prob = prob * 0.2
                activityToday = "Sleep"
                activityList.append("Sleep")
            else:
                prob = prob * 0.7
                activityToday = "Run"
                activityList.append("Run")
        i += 1  
    print("Possible states: " + str(activityList))
    print("End state after "+ str(days) + " days: " + activityToday)
    print("Probability of the possible sequence of states: " + str(prob))

# Function that forecasts the possible state for the next 2 days
activity_forecast(2)


Start state: Sleep
Possible states: ['Sleep', 'Run', 'Run']
End state after 2 days: Run
Probability of the possible sequence of states: 0.3


# Minimum note duration

- Changes in note durations
- minumum note duration is 0.5s
- maximum note duration is 8 seconds
- To be modeled using a markov chain

In [99]:
note_durations = [0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
# The statespace
states = ["VS","S","M","L","VL"]
# Possible sequences of events
transitionName = [["VSVS","VSS","VSM","VSL","VSVL"],["SVS","SS","SM","SL","SVL"],["MVS","MS","MM","ML","MVL"],["LVS","LS","LM","LL","LVL"],["VLVS","VLS","VLM","VLL","VLVL"]]
# Probabilities matrix (transition matrix)
#TODO: Load transition matrix from each theme
transitionMatrix = np.array([[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2],[0.2,0.2,0.2,0.2,0.2]])
assert sum(transitionMatrix[0])==1
assert sum(transitionMatrix[1])==1
assert sum(transitionMatrix[2])==1
assert sum(transitionMatrix[3])==1
assert sum(transitionMatrix[4])==1


def get_length_sequence(initial_state, n_steps, seed="NA"):
    sequence = [initial_state]
    if seed != "NA":
        np.random.seed(seed)
    for i in range(n_steps-1):
        transition_probs = transitionMatrix[states.index(sequence[-1])]
        next_state = np.random.choice(states, p=transition_probs)
        sequence.append(next_state)
    return sequence

def get_durations(total_length, sequence):
    length_reqd = total_length
    durations = []
    for i in range(len(sequence)):
        if i == len(sequence)-1:
            if length_reqd < 3:
                durations.append(length_reqd)
                break
        if sequence[i] == "VS":
            durations.append(rm.uniform(0.5,1))
        elif sequence[i] == "S":
            durations.append(rm.uniform(1,2))
        elif sequence[i] == "M":
            durations.append(rm.uniform(2,3))
        elif sequence[i] == "L":
            durations.append(rm.uniform(3,5))
        elif sequence[i] == "VL":
            durations.append(rm.uniform(5,8))
        length_reqd -= durations[i]
        # print(length_reqd)
        # print(durations)
        if sum(durations) > total_length:
            durations = durations[:-1]
            durations.append(total_length-sum(durations))
            return durations
    return durations

In [100]:
get_length_sequence("VL",20)

['VL',
 'M',
 'M',
 'VS',
 'L',
 'M',
 'L',
 'VL',
 'VL',
 'VL',
 'VS',
 'S',
 'L',
 'VS',
 'M',
 'VS',
 'VS',
 'VS',
 'L',
 'S']

In [109]:
durs = get_durations(20,get_length_sequence("VL",20))
print(durs)
print(sum(durs))

[5.2937271081268324, 1.1819885637992447, 0.5433863438972469, 6.378951745295597, 0.7841233867891114, 2.3879217089372413, 1.8492768135184037, 0.516635769890291, 1.0639885597460292]
20.0


# Note Density
- Number of simultaneous notes  
- Ranges from 1 to 5  
- Modeled using a markov chain  

# Melodic Range
- Number of octaves to select from (Range: 1 - 3)
- Specific octave to use (Changes from one timbre to another)
- Modeled using random sampling with probabilities

# Loudness
- Scaling factor for the final output (Range: 0 - 1)
- Initial and final values to be selected and connected using a Gensound Linear or Exponential object

# Melodic Direction
- Ascending or descending based on the selected segment